In [1]:
import weave
import os
from pinecone import Pinecone
from llama_index.vector_stores.pinecone import PineconeVectorStore
from dotenv import load_dotenv
load_dotenv()
from app.engine import get_chat_engine

/home/alirio/.cache/pypoetry/virtualenvs/app-7J7vEg79-py3.12/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone()
index_name = os.getenv("PINECONE_INDEX_NAME")
index = pc.Index(index_name)
index.describe_index_stats()
vector_store = PineconeVectorStore(
    pinecone_index=index,
)

In [3]:
import pandas as pd
qa_df = pd.read_csv("qa_current.csv")
# qa_df.head()


In [4]:
from llama_index.core.settings import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model_name = "text-embedding-3-large"

Settings.embed_model = OpenAIEmbedding(
    model=embed_model_name,
    embed_batch_size=100,
    max_retries=25,
    timeout=180,
    reuse_client=True,
    dimensions=3072,
)


In [5]:
from llama_index.llms.openai import OpenAI

max_tokens = os.getenv("LLM_MAX_TOKENS")

Settings.llm = OpenAI(
    model="gpt-4o-mini",
    temperature=float(0),
    max_tokens=int(max_tokens) if max_tokens is not None else None,
)
# Settings.llm.model

In [6]:
client = weave.init('pathway_chatbot2')

Logged in as Weights & Biases user: mie21001.
View Weave data at https://wandb.ai/mie21001-byu/pathway_chatbot2/weave


In [7]:
async def generate_answer(question: str, chat_engine):
    response = await chat_engine.achat(question)
    chat_engine.reset()
    return response
    

In [8]:
@weave.op()
def save_retrieved_content(question, response):
    retrieved = "\n\n".join(
        [
            f"node_id: {idx+1}\n{node.metadata['url']}\n{node.text}"
            for idx, node in enumerate(response.source_nodes)
        ]
    )
    return retrieved

In [10]:
from test_tools import evaluate_response

messages = ""
new_data = []
chat_engine = get_chat_engine()


for index, row in qa_df.iterrows():
    question = row["Question"]
    ideal_answer = row["Ideal Answer"]
    response = await generate_answer(question, chat_engine)
    retrieved, call = save_retrieved_content.call(question, response)
    call.feedback.add_reaction("👍")


Condensed question: How many credits for a Bachelors degree?
Context: We have 9 nodes:
node_id: 1
text: Program Strategies / Bachelor's Degree (90–96 credit hours) Our three-year bachelor's degrees are made up of three certificates, plus general courses. These degrees may help you progress within your career and earn additional income. In addition, courses taken as part of certificates and an associate degree count toward bachelor's degrees. View Bachelor's Degrees

node_id: 2
text: Web Development / Questions / What is the difference between a certificate, an associate degree, and a bachelor's degree? * A certificate is 12–15 credits and focuses on one area of study. * An associate degree is 60 credits and includes general studies. * A bachelor’s degree is 90 credits and includes a major field of study as well as general studies.

node_id: 3
text: Earn a bachelor's degree in three years through / Fewer credits. 100% online. Through , BYU-Idaho and Ensign College offer 90–96 credit deg

CancelledError: 

### Feedback
#### Get the trace

In [ ]:
client = weave.init("mie21001-byu/pathway_chatbot")
call = client.get_call("0192b507-bb0d-7313-84ec-c94308a72251")

#### Add reaction

In [ ]:
call.feedback.add_reaction("👍")

#### Add a comment

In [ ]:
call.feedback.add_note("This a test!")

#### Delete the feedback

In [19]:
for f in call.feedback:
    # print(f.id)
    # print(f.feedback_type)
    # print(f.payload)
    call.feedback.purge(f.id)

In [11]:
# Example
# call.feedback.purge("0192ba38-6f96-70a0-b0a9-538b53de8af1")

In [ ]:
# https://wandb.ai/mie21001-byu/pathway_chatbot/weave/calls?filter=%7B%22opVersionRefs%22%3A%5B%22weave%3A%2F%2F%2Fmie21001-byu%2Fpathway_chatbot%2Fop%2Fsave_retrieved_content%3A*%22%5D%7D&cols=%7B%22attributes.weave.client_version%22%3Afalse%2C%22attributes.weave.source%22%3Afalse%2C%22attributes.weave.os_name%22%3Afalse%2C%22attributes.weave.os_version%22%3Afalse%2C%22attributes.weave.os_release%22%3Afalse%2C%22attributes.weave.sys_version%22%3Afalse%2C%22feedback%22%3Atrue%2C%22status%22%3Afalse%2C%22inputs.question%22%3Atrue%2C%22inputs.response.is_dummy_stream%22%3Afalse%2C%22inputs.response.response%22%3Atrue%2C%22inputs.response.source_nodes%22%3Afalse%2C%22inputs.response.sources%22%3Afalse%2C%22output%22%3Atrue%2C%22wb_user_id%22%3Afalse%2C%22started_at%22%3Atrue%2C%22tokens%22%3Afalse%2C%22cost%22%3Afalse%2C%22latency%22%3Afalse%7D&pin=%7B%22right%22%3A%5B%22feedback%22%5D%2C%22left%22%3A%5B%22op_name%22%5D%7D

#### Implement in the chatbot

In [ ]:

@weave.op()
def simple_operation(input_value):
    return f"Processed {input_value}"

# Execute the operation and retrieve the result and call ID
result, call = simple_operation.call("example input")
call_id = call.id